# quick 'n dirty solver for sunshine heavy industries.

game is good, [buy it now on steam](https://store.steampowered.com/app/1542810/Sunshine_Heavy_Industries/).

coming in second place in the leaderboard is no fun. use this notebook to change that.

## notes

currently only solves for basic threshold constraints, manuverability, fuel & heatsink adjacency, and total-area constraints

todo: armor requirements, cloaking area constraints, energy area constraints and x-width/y-width constraints


## to use

cost table extracted as of halloween patch (10/30/2021-ish or so). packed asar needs to be extracted and then the obfuscated node module run in order to generate the full `costs.json` via `yarn generate` in the root node project.

run this notebook w/ the relevant constraints.
requires everything in `requirements.txt`. Also needs GLPK binary as mixed-integer linear programming solver.


In [1]:
from json import loads
import numpy as np
import pandas as pd
from pyomo.environ import *

costs = None
with open("costs.json") as f:
    costs = list(loads(f.read()).items())

print(f"Parts in DB: {len(costs)}")

Parts in DB: 108


In [2]:
def prop_vector(prop_name, default=0):
    return np.array([data.get(prop_name, default) for _, data in costs])

names = [n for n, _ in costs]
hitboxes = [np.array(data.get('hitbox', np.ones([data.get('width'),data.get('height')]))) for _, data in costs]
area = np.array([np.count_nonzero(hb) for hb in hitboxes])

borders = np.array([[not x in ob for x in range(4)] for ob in prop_vector('mustBeUnobstructed', [])])

perimeter = \
    prop_vector('height') * borders[:,0] + \
    prop_vector('width') * borders[:, 1] + \
    prop_vector('height') * borders[:, 2] + \
    prop_vector('width') * borders[:, 3]

data = pd.DataFrame({
    'part': names,
    'mass': area,
    'thrust': area * prop_vector('thrust'),
    'cost': area * prop_vector('cost'),
    'energy': area * prop_vector('energy'),
    'energyPiece': (prop_vector('energy') > 0) * 1,
    'fuel': area * prop_vector('fuel'),
    'firepower': area * prop_vector('firepower'),
    'passenger': area * prop_vector('quarters'),
    'command': area * prop_vector('command'),
    'agility': area * prop_vector('agility'),
    'cargo': area * prop_vector('cargo'),
    'fuelAdjacency': prop_vector('pump') * perimeter - prop_vector('mustTouchFuelPump'),
    'fuelPiece': prop_vector('pump'),
    'sinkPiece': prop_vector('heatsink'),
    'sinkAdjacency': prop_vector('heatsink') * perimeter - prop_vector('heat'),
    'fuelPiece': prop_vector('pump'),
})

# pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

data.set_index('part', inplace=True)
data

,mass,thrust,cost,energy,energyPiece,fuel,firepower,passenger,command,agility,cargo,fuelAdjacency,fuelPiece,sinkPiece,sinkAdjacency
part,,,,,,,,,,,,,,,
antenna_1x2,2,0.0,100.0,0.0,0,0,0,0,0,0,0,0,0,0,0
antenna_1x4,4,0.0,400.0,0.0,0,0,0,0,0,0,0,0,0,0,0
antenna_1x5,5,0.0,300.0,0.0,0,0,0,0,0,0,0,0,0,0,0
bunk_1x1,1,0.0,20.0,0.0,0,0,0,1,0,0,0,0,0,0,0
bunk_2x1,2,0.0,40.0,0.0,0,0,0,2,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
tank_pod_3x3,9,0.0,315.0,0.0,0,9,0,0,0,0,0,-1,0,0,0
tractorbeam_2x3,4,0.0,120.0,-8.0,0,0,0,0,0,0,0,0,0,0,-1
turbine_2x2,4,0.0,900.0,40.0,1,0,0,0,0,0,0,0,0,1,8


In [3]:
# normal constraints
FUEL = 14
THRUST = 32
FIREPOWER = 0
PASSENGER = 20
COMMAND = 4
CARGO = 18
MANUVERABILITY = 0.38

# dimension constraints
AREA_COST = 10

# arrangement tweaks
# (start at 0, increase if config is physically impossible)
FUEL_PIECE = 2
SINK_PIECE = 2
ENERGY_PIECE = 0
FUEL_ADJACENCY = 1 # number of blocked fuel pump ports

model = ConcreteModel()
model.x = Var(names, domain=NonNegativeIntegers)
model.total_cost = Objective(
    expr = sum(data.loc[i].cost * model.x[i] + data.loc[i].mass * AREA_COST * model.x[i]  for i in names),
    sense = minimize
)

# thresholds
model.fuel = Constraint(
    expr = sum(data.loc[i].fuel * model.x[i] for i in names) >= FUEL
)
model.thrust = Constraint(
    expr = sum(data.loc[i].thrust * model.x[i] for i in names) >= THRUST
)
model.firepower = Constraint(
    expr = sum(data.loc[i].firepower * model.x[i] for i in names) >= FIREPOWER
)
model.passenger = Constraint(
    expr = sum(data.loc[i].passenger * model.x[i] for i in names) >= PASSENGER
)
model.command = Constraint(
    expr = sum(data.loc[i].command * model.x[i] for i in names) >= COMMAND
)
model.cargo = Constraint(
    expr = sum(data.loc[i].cargo * model.x[i] for i in names) >= CARGO
)
model.energy = Constraint(
    expr = sum(data.loc[i].energy * model.x[i] for i in names) >= 0
)

# adjacencies
model.fuelAdjacency = Constraint(
    expr = sum(data.loc[i].fuelAdjacency * model.x[i] for i in names) >= FUEL_ADJACENCY
)
model.sinkAdjacency = Constraint(
    expr = sum(data.loc[i].sinkAdjacency * model.x[i] for i in names) >= 0
)

# tweaks
model.fuelPiece = Constraint(
    expr = sum(data.loc[i].fuelPiece * model.x[i] for i in names) >= FUEL_PIECE
)

model.sinkPiece = Constraint(
    expr = sum(data.loc[i].sinkPiece * model.x[i] for i in names) >= SINK_PIECE
)

model.energy_piece = Constraint(
    expr = sum(data.loc[i].energyPiece * model.x[i] for i in names) >= ENERGY_PIECE
)


# agility / mass >= manuverability
model.agility = Constraint(
    expr = sum((data.loc[i].agility - MANUVERABILITY * data.loc[i].mass) * model.x[i] for i in names) >= 0
)

solver=SolverFactory('glpk', executable="/usr/local/bin/glpsol")
results = solver.solve(model)
print(results)
model.display()


res = []
for i in model.x:
    if(model.x[i].value > 0):
        res.append([str(model.x[i])[2:-1], int(model.x[i].value)])
        
df = pd.DataFrame(res, columns=['Part','Qty'])
df.style.hide_index() 


Problem: 
- Name: unknown
  Lower bound: 3865.0
  Upper bound: 3865.0
  Number of objectives: 1
  Number of constraints: 14
  Number of variables: 109
  Number of nonzeros: 258
  Sense: minimize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 79
      Number of created subproblems: 79
  Error rc: 0
  Time: 0.013914823532104492
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Model unknown

  Variables:
    x : Size=108, Index=x_index
        Key                 : Lower : Value : Upper : Fixed : Stale : Domain
                antenna_1x2 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                antenna_1x4 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                antenna_1x5 :     0 :   0.0 :  None : False : False : NonNegativeIntegers
                   bunk_1x1 :     0 :  16.0 :  None : False : False : NonNegativeIntegers
                   

Part,Qty
bunk_1x1,16
cab_2x2,1
cargo_rad_3x2,3
engine_3x1,1
engine_5x1,1
enginedirty_4x2,2
fin_2x2,1
heat_vent_1x1,2
pump_1x1,2
tank_2x1,1
